**<h3><center>EN2550 Exercise 8 - Plotting Epipolar Lines</center></h3>**
*Name - Abeysinghe W.A.M.S.Y*
<br>
*Index no - 190014F*
<br>

All the imports are done at the beginning. 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from plyfile import PlyData, PlyElement # Open3d is the standard

The Middlebury Multi-View Stereo dataset contains calibrated image. Read the webpage
and download the TempleSparseRing 1 dataset. https://vision.middlebury.edu/mview/data/ 

**Q1**. Read in the templeSR_par.txtg and populate the parameters of Ki Ri and ti for i = i,2.

In [4]:
f = open(r'templeSparseRing/templeSR_par.txt', 'r')                                                 
assert f is not None
n = int(f.readline())

#Reading the information of the first page
l = f.readline().split()
im1_fn = l[0]
K1 = np.array([float(i) for i in l[1:10]]).reshape((3,3))
R1 = np.array([float(i) for i in l[10:19]]).reshape((3,3))
t1 = np.array([float(i) for i in l[19:22]]).reshape((3,1))

#Reading the information of the second page
l = f.readline().split()
im2_fn = l[0]
K2 = np.array([float(i) for i in l[1:10]]).reshape((3,3))
R2 = np.array([float(i) for i in l[10:19]]).reshape((3,3))
t2 = np.array([float(i) for i in l[19:22]]).reshape((3,1))

#Read the two images and show
im1 = cv.imread(r'templeSparseRing/' + im1_fn, cv.IMREAD_COLOR)
im2 = cv.imread(r'templeSparseRing/' + im2_fn, cv.IMREAD_COLOR)
assert im1 is not None
assert im2 is not None





**Q2**. Compute the camera matrices Ki for i = i,2.

**Q3**. Plot the epipolar line corresponding to the point x = [130 115 1]T in image 1.

**Q4**. Detect and plot the features (e.g., SIFT) and plot the corresponding epipolar lines.